# Building a Question-Answering System with Hugging Face Transformers

Hugging Face’s transformers package contains several pretrained BERT models already fine-tuned for specific tasks. One example is the “minilm-uncased-squad2” model, which was trained with Stanford’s [SQuAD 2.0](https://rajpurkar.github.io/SQuAD-explorer/) dataset to answer questions by extracting text from documents. To get a feel for what models like this one can accomplish, let’s use it to build a simple question-answering system.

Begin by using the following statements to load a pretrained MiniLM model from the Hugging-Face hub and a tokenizer to tokenize text input to the model. Then compose a pipeline from them. The first time you run this code, you'll experience a momentary delay while the pretrained weights are downloaded. After that, loading will be fast because the weights are cached:

In [1]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, pipeline

id = 'deepset/minilm-uncased-squad2'
tokenizer = AutoTokenizer.from_pretrained(id)
model = TFAutoModelForQuestionAnswering.from_pretrained(id, from_pt=True)
pipe = pipeline('question-answering', model=model, tokenizer=tokenizer)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


Hugging Face stores weights for this particular model in PyTorch format. The `from_pt=True` parameter converts the weights to TensorFlow format. It's not trivial to convert neural-network weights from one format to another, but the Hugging-Face library reduces it to a simple function parameter.

Now use the pipeline to answer a question by extracting text from a paragraph:

In [2]:
question = 'What does NLP stand for?'

context = 'Natural Language Processing, or NLP, encompasses a variety of \
           activities, including text classification, keyword and topic \
           extraction, text summarization, and language translation. The \
           accuracy of NLP models has improved in recent years for a variety \
           of reasons, not the least of which are newer and better ways of \
           converting words and sentences into dense vector representations \
           that incorporate context, and a relatively new neural-network \
           architecture called the transformer that can zero in on the most \
           meaningful words and even differentiate between multiple meanings \
           of the same word.'

pipe(question=question, context=context)

{'score': 0.9793193340301514,
 'start': 0,
 'end': 27,
 'answer': 'Natural Language Processing'}

Is the answer accurate? A human could easily read the paragraph and come up with the same answer, but the fact that a deep-learning model can do it indicates that the model displays some level of reading comprehension. Observe that the output contains the answer to the question as well as a confidence score and the starting and ending indexes of the answer in the paragraph.

Now try it again with a different question and context:

In [3]:
question = 'When was TensorFlow released?'

context = 'Machine learning isn\'t hard when you have a properly engineered \
           dataset to work with. The reason it\'s not hard is libraries such as \
           Scikit-learn and ML.NET, which reduce complex learning algorithms to \
           a few lines of code. Deep learning isn’t difficult, either, thanks to \
           libraries such as the Microsoft Cognitive Toolkit (CNTK), Theano, \
           and PyTorch. But the library that most of the world has settled on \
           for building neural networks is TensorFlow, an open-source framework \
           created by Google that was released under the Apache License 2.0 in \
           2015.'

pipe(question=question, context=context)['answer']

'2015'

Repeat this process with another question and context from which to extract an answer:

In [4]:
question = 'Is Keras part of TensorFlow?'

context = 'The learning curve for TensorFlow is rather steep. Another library \
           named Keras provides a simplified Python interface to TensorFlow \
           and has emerged as the Scikit of deep learning. Keras is all about \
           neural networks. It began life as a stand-alone project in 2015 \
           but was integrated into TensorFlow in 2019. Any code that you write \
           using TensorFlow’s built-in Keras module ultimately executes in \
           (and is optimized for) TensorFlow. Even Google recommends using the \
           Keras API.'

pipe(question=question, context=context)['answer']

'integrated into TensorFlow in 2019'

Perform one final test using the same context as before but a different question:

In [5]:
question = 'Is it better to use Keras or TensorFlow to build neural networks?'
pipe(question=question, context=context)['answer']

'Keras'

The questions posed here were hand-selected to highlight the model's capabilities. It's not difficult to come up with questions that the model can't answer. Nevertheless, you have proved the principle that a pretrained BERT model fine-tuned on SQuAD 2.0 can answer straightforward questions from passages of text presented to it.

## Add a retriever

The reader's job is to extract answers from text containing answers. An equally important component of a QA pipeline is a retriever that selectively fetches text from a database or other document store and presents it to the reader. In real life, companies use tools such as [Haystack](https://haystack.deepset.ai/overview/intro) to build production-ready QA pipelines. Haystack provides highly scalable retrievers that operate against a variety of document stores, and it allows Hugging Face transformers such as MiniLM to serve as readers.

Let's implement a simple retriever using a pretrained transformer model that converts paragraphs of text into vectors of floating-point numbers. We'll judge which paragraphs of text might contain answers by using dot products to measure the similarity between a vectorized question and vectorized versions of each paragraphs in a corpus of text. Begin by loading the dataset, which contains 25 paragraphs of text:

In [6]:
import pandas as pd

data = pd.read_csv('Data/passages.csv', header=None)
pd.set_option('display.max_colwidth', None)
data.head()

,0
0,"Machine learning isn’t hard when you have a properly engineered dataset to work with. The reason it’s not hard is libraries such as Scikit-learn and ML.NET, which reduce complex learning algorithms to a few lines of code. Deep learning isn’t difficult, either, thanks to libraries such as the Microsoft Cognitive Toolkit (CNTK), Theano, and PyTorch. But the library that most of the world has settled on for building neural networks is TensorFlow, an open-source framework created by Google that was released under the Apache License 2.0 in 2015."
1,"TensorFlow isn’t limited to building neural networks. It is a framework for performing fast mathematical operations at scale using tensors, which are generalized versions of arrays. Tensors can represent scalar values (0-dimensional tensors), vectors (1D tensors), matrices (2D tensors), and so on. A neural network is basically a workflow for transforming tensors. The 3-layer perceptron featured in the previous chapter takes a 1D tensor containing two values as input, transforms it into a 1D tensor containing three values, and produces a 0D tensor as output. TensorFlow lets you define directed graphs that in turn define how tensors are computed. And unlike Scikit, it supports GPUs."
2,The learning curve for TensorFlow is rather steep. Another library named Keras provides a simplified Python interface to TensorFlow and has emerged as the Scikit of deep learning. Keras is all about neural networks. It began life as a stand-alone project in 2015 but was integrated into TensorFlow in 2019. Any code that you write using TensorFlow’s built-in Keras module ultimately executes in (and is optimized for) TensorFlow. Even Google recommends using the Keras API.
3,"Keras offers two APIs for building neural networks: a sequential API and a functional API. The former is simpler and is sufficient for most neural networks. The latter is useful in more advanced scenarios such as networks with multiple outputs – for example, a classification output and a regression output, which is common in neural networks that perform object detection – or shared layers."
4,"It’s not difficult to use Scikit-learn to build machine-learning models that analyze text for sentiment, identify spam, and classify text in other ways. But today, state-of-the-art text classification is most often performed with neural networks. You already know how to build neural networks that accept numbers and images as input. Let’s build on that to learn how to construct deep-learning models that process text – a segment of deep learning known as natural language processing, or NLP for short."


Next, load a pretrained model (and a tokenizer to go with it) that is adept at vectorizing text in such a way that the similarity between two vectorized samples can be quantified:

In [7]:
from transformers import TFAutoModel

bert_id = 'sebastian-hofstaetter/distilbert-dot-margin_mse-T2-msmarco'
bert_tokenizer = AutoTokenizer.from_pretrained(bert_id) 
bert_model = TFAutoModel.from_pretrained(bert_id, from_pt=True)

All PyTorch model weights were used when initializing TFDistilBertModel.

All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Define a function that uses the model to vectorize each of the paragraphs in the dataset. Then use the function to convert each paragraph into a vector (array) of 768 floating-point values and store the vectors in a variable named `vectorized_contexts`:

In [8]:
def vectorize_text(text):
    tokenized_text = bert_tokenizer(text, return_tensors='tf')
    vectorized_text = bert_model(tokenized_text)[0][:, 0, :][0]
    return vectorized_text

contexts = data[0]
vectorized_contexts = contexts.apply(vectorize_text)

Define a function that vectorizes a question and computes its similarity to each of the paragraphs stored in `vectorized_contexts`. Similarity is computed by taking the dot product of the question and each sample. The higher the dot product, the greater the similarity. `get_best_contexts` sorts the similarity scores in descending order and returns the indexes of the top `max_matches` results:

In [9]:
import numpy as np

def get_best_contexts(query, contexts, max_matches=3):
    scores = pd.Series(dtype='object')
    tokenized_query = bert_tokenizer(query, return_tensors='tf')
    vectorized_query = bert_model(tokenized_query)[0][:, 0, :][0]
    
    for idx, item in contexts.iteritems():
        score = np.dot(vectorized_query, item)
        scores = scores.append(pd.Series(score), ignore_index=True)

    sorted_scores = scores.sort_values(ascending=False)[:max_matches]
    return list(sorted_scores.index)

Now use `get_best_contexts` to identify the three paragraphs that are most likely to contain an answer to the question "How many versions of YOLO are there?"

In [10]:
question = 'How many versions of YOLO are there?'
indexes = get_best_contexts(question, vectorized_contexts)

for idx in indexes:
    print(f'{contexts[idx]}\n')

There are five versions of YOLO referred to as YOLOv1 through YOLOv5. There are also variations such as PP-YOLO and YOLO9000. YOLOv3 processes the image using a 13x13 grid, a 26x26 grid, and a 52x52 grid so it can detect objects large and small. It borrows the anchor-box concept from Faster R-CNN to predict nine bounding boxes per cell. YOLO’s primary weakness is that it has difficulty detecting small objects that are close together. This is a consequence of the fact that a given cell can only predict one class. 

While the R-CNN family of object-detection systems delivers unparalleled accuracy, it leaves something to be desired when it comes to real-time object detection of the type required by, say, self-driving cars. A paper entitled “You Only Look Once: Unified, Real-Time Object Detection” published in 2015 proposed an alternative to R-CNNs known as YOLO that revolutionized the way data scientists think about object detection. 

Until a few short years ago, most NMT models, includi

Submit the three paragraphs identified by `get_best_contexts` to the QA model to extract up to three answers. Show the answers, but skip empty answers — ones for which the starting and ending indexes are identical:

In [11]:
for idx in indexes:
    output = pipe(question=question, context=contexts[idx], handle_impossible_answer=True)
    
    if output['start'] != output['end']:
        print(output)

{'score': 0.4768390357494354, 'start': 10, 'end': 14, 'answer': 'five'}


Define a function that takes a question, a set of paragraphs, and the vectorized versions of those paragraphs as inputs and displays the answer or answers to the question. Then ask a question and show the results:

In [12]:
def show_answers(question, contexts, vectorized_contexts):
    indexes = get_best_contexts(question, vectorized_contexts)
    
    for idx in indexes:
        output = pipe(question=question, context=contexts[idx], handle_impossible_answer=True)

        if output['start'] != output['end']:
            print(f'{output["answer"]} ({output["score"]:.1%})')

question = 'What type of neural network supports instance segmentation?'
show_answers(question, contexts, vectorized_contexts)

Mask R-CNNs (84.5%)


Do it again, this time with a different question:

In [13]:
question = 'What is YOLO\'s primary weakness?'
show_answers(question, contexts, vectorized_contexts)

it has difficulty detecting small objects that are close together (52.9%)


Try a third question and see if the model is able to come up with an answer:

In [14]:
question = 'Is TensorFlow difficult to learn?'
show_answers(question, contexts, vectorized_contexts)

TensorFlow (13.0%)
The learning curve for TensorFlow is rather steep (26.9%)


Building a robust QA system that can answer the bulk of the questions presented to it on a particular topic obviously requires a much larger corpus of text than the 25 passages you used here. And the larger the corpus of text, the more such a system would benefit from a product such as Haystack. The key, however, is a reading-comprehension model that can make sense of the questions asked of it. For that, pretrained models from Hugging Face stand ready to get you up and running quickly.